In [1]:
import pandas as pd
import tensorflow as tf 
import tensorflow_recommenders as tfrs 
from typing import Dict, Text

2024-09-07 12:41:19.298982: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-07 12:41:19.495095: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-07 12:41:19.653529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-07 12:41:19.840423: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-07 12:41:19.840861: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-07 12:41:20.184757: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
# loading the data sources
user_movie_title_rating_data = pd.read_csv('/workspaces/imdb_movie_deep_learning_tf_recommender/research/sliced_user_id_and_movie_title_data.csv')
# user_rating_data = pd.read_csv('/Users/eugene/Personal_Projects/Real_ML_Project/imdb_dataset/title.user-rating.csv')
# movie_data = pd.read_csv('/Users/eugene/Personal_Projects/Real_ML_Project/imdb_dataset/title.basics.csv')
# average_rating_data =  pd.read_csv('/Users/eugene/Personal_Projects/Real_ML_Project/imdb_dataset/title.ratings.csv')
# basics_data = pd.read_csv('/Users/eugene/Personal_Projects/Real_ML_Project/imdb_dataset/name.basics.csv')
# principals_data = pd.read_csv('/Users/eugene/Personal_Projects/Real_ML_Project/imdb_dataset/title.principals.csv')

In [3]:
# visualize data
user_movie_title_rating_data.head()
# # rename the tconst to movieID
# movie_data.rename(columns={'tconst': 'movieID'}, inplace=True)

# # performing a merge operation to expand the user rating data with movie titles
# user_rating_data_with_movies_title = user_rating_data.merge(movie_data, how='left', on='movieID')
# user_rating_data_with_movies_title = user_rating_data_with_movies_title[['userID', 'movieID','originalTitle', 'rating', 'review date']]


,userID,movieID,originalTitle,rating,review date
0,ur4592644,tt0120884,When the Light Comes,10,16 January 2005
1,ur3174947,tt0118688,Batman & Robin,3,16 January 2005
2,ur3780035,tt0387887,Bottom Live 2003: Weapons Grade Y-Fronts Tour,8,16 January 2005
3,ur4592628,tt0346491,Alexander,1,16 January 2005
4,ur3174947,tt0094721,Beetlejuice,8,16 January 2005


In [4]:
# # Select basic feature for modeling 
# user_id_and_movie_title_data = user_rating_data_with_movies_title[['userID', 'originalTitle','rating']]
# user_id_and_movie_title_data.head()

,userID,originalTitle,rating
0,ur4592644,When the Light Comes,10
1,ur3174947,Batman & Robin,3
2,ur3780035,Bottom Live 2003: Weapons Grade Y-Fronts Tour,8
3,ur4592628,Alexander,1
4,ur3174947,Beetlejuice,8


In [5]:
# # experimental step of extracting just a slice of our data for testing purposes
# sliced_user_id_and_movie_title_data = user_id_and_movie_title_data[0:10000]

In [4]:
# create a new movie title list that enforces type uniformity in its values
# this step came up because of inconsitency when creating movie title vocabulary
updated_movie_titles = []
for i in user_movie_title_rating_data['originalTitle']:
    if type(i) != 'str':
        a = str(i)
        updated_movie_titles.append(a)
    else:
        updated_movie_titles.append(i)

# assign the new list to the original title column of the sliced user id and movie title data
user_movie_title_rating_data['originalTitle'] = updated_movie_titles

In [5]:
# obtain the tensor form for our user_id_and_movie_title dataset. 
# This is essential during the model fit operation
user_movie_title_rating_data_tensor = tf.data.Dataset.from_tensor_slices({'userID': tf.cast(user_movie_title_rating_data['userID'].values, tf.string),
                                    'originalTitle': tf.cast(user_movie_title_rating_data['originalTitle'].values, tf.string),
                                    'rating': tf.cast(user_movie_title_rating_data['rating'].values, tf.int64)})

In [8]:
# shuffle the user_id_movie_title dataset
tf.random.set_seed(42)
user_movie_title_rating_data_tensor_shuffled= user_movie_title_rating_data_tensor.shuffle(50000, seed=42, reshuffle_each_iteration=False)

In [9]:
# split the data into train and test
train = user_movie_title_rating_data_tensor_shuffled.take(40000)
test = user_movie_title_rating_data_tensor_shuffled.skip(40000).take(10000)

In [10]:
#get the unique user ids and movie titles for the purpose of embedding
unique_user_ids = user_movie_title_rating_data['userID'].unique()
unique_movie_titles = user_movie_title_rating_data['originalTitle'].unique()

In [11]:
# Define the ranking model architecture
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, userID, originalTitle):

    user_embedding = self.user_embeddings(userID)
    movie_embedding = self.movie_embeddings(originalTitle)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [12]:
class IMDBModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features) -> tf.Tensor:
    return self.ranking_model(features["userID"], features["originalTitle"]  )

  def compute_loss(self, features, training=False) -> tf.Tensor:
    rating_predictions = self.ranking_model(features["userID"], features["originalTitle"]  )

    # The task computes the loss and the metrics.
    return self.task(labels=features["rating"], predictions=rating_predictions)

In [13]:
#fitting the model
model = IMDBModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [15]:
# Then shuffle, batch, and cache the training and evaluation data.
cached_train = train.shuffle(40000).batch(1000).cache()
cached_test = test.batch(200).cache()

In [16]:
# To get the full stack trace
tf.keras.config.disable_traceback_filtering()
# Training the model
model.fit(cached_train, epochs=5)

Epoch 1/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 11.7220 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 4.2721 - total_loss: 11.7220
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7.2188 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.7642 - total_loss: 7.2188
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.5762 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.4526 - total_loss: 5.5762
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.0118 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.1011 - total_loss: 4.0118
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.9791 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 1.7669 - total_loss: 2.9791


In [17]:
# As the model trains, the loss is falling and the RMSE metric is improving.
# Finally, we can evaluate our model on the test set:
model.evaluate(cached_test, return_dict=True)

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.4826 - regularization_loss: 0.0000e+00 - root_mean_squared_error: 2.8929 - total_loss: 8.4826


{'loss': 7.197841167449951,
 'regularization_loss': 0.0,
 'root_mean_squared_error': 2.916900873184204,
 'total_loss': 7.197841167449951}

In [18]:
#Now we can test the ranking model by computing predictions for a set of movies 
# and then rank these movies based on the predictions:
test_ratings = {}
test_movie_titles = ["Batman & Robin", "Beetlejuice", "When the Light Comes"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "userID": pd.Series(["ur3174947"]).to_numpy(),
      "originalTitle": pd.Series([movie_title]).to_numpy()
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
When the Light Comes: [[10.786108]]
Beetlejuice: [[8.073874]]
Batman & Robin: [[4.122521]]


In [19]:
user_rand = unique_user_ids[1]
test_rating = {}
for m in test.take(5):
    test_rating[m["originalTitle"].numpy()]=RankingModel()(tf.convert_to_tensor([user_rand]),tf.convert_to_tensor([m["originalTitle"]]))

2024-09-07 12:52:19.270915: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [20]:
print("Top 5 recommended products for User {}: ".format(user_rand))
for m in sorted(test_rating, key=test_rating.get, reverse=True):
    print(m.decode())

Top 5 recommended products for User ur3174947: 
High Society
Looking for Richard
American History X
Backroads
Chasing Liberty
